# Using DFTpy mixer

In [1]:
import _qepy
import qepy
import importlib

In [2]:
import numpy as np
from qepy.driver import Driver
from qepy.io import QEInput

In [3]:
from ase.io.trajectory import Trajectory
from ase.lattice.hexagonal import Graphene
from ase import Atoms
import matplotlib.pyplot as plt

In [4]:
qe_options = {
    '&control': {
        'calculation': "'scf'",
        'pseudo_dir': "'./'"
    },
    '&system': {
        'ibrav' : 0,
        'degauss': 0.005,
        'ecutwfc': 30,
        'nat': 1,
        'ntyp': 1,
        'occupations': "'smearing'",
        'input_dft' : "'PBE'"
    },
    '&electrons': {
        'conv_thr' : 1e-8
    },
    'atomic_positions crystal': ['Al    0.0  0.0  0.0'],
    'atomic_species': ['Al  26.98 Al_ONCV_PBE-1.2.upf'],
    'k_points automatic': ['4 4 4 0 0 0'],
    'cell_parameters angstrom':[
        '0.     2.025  2.025',
        '2.025  0.     2.025',
        '2.025  2.025  0.   '],
}

In [5]:
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp.out')

In [6]:
for i in range(60):
    driver.diagonalize()
    driver.mix()
    converged = driver.check_convergence()
    print ('Iter: ',i,' - Conv: ', driver.get_scf_error())
    if converged : break
driver.calc_energy()

Iter:  0  - Conv:  0.07423450252710213
Iter:  1  - Conv:  0.0014686764583955348
Iter:  2  - Conv:  3.4654773793272354e-05
Iter:  3  - Conv:  6.680306265302417e-08
Iter:  4  - Conv:  2.5412863642447476e-09


-137.91449178319718

In [7]:
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp.2.out')

In [8]:
from dftpy.functional import Hartree
from dftpy.mixer import Mixer
driver.mixer = Mixer(scheme='pulay')

In [9]:
rho = driver.data2field(driver.get_density().copy())
for i in range(10):
    driver.diagonalize()
    #
    rho_new = driver.data2field(driver.get_density().copy())
    error = Hartree.compute(rho_new-rho).energy
    print ('Iter: ',i,' - Conv: ', error)
    if error < 1e-8:
        driver.end_scf()
        break
    #
    rho = driver.mixer(rho, rho_new, coef=0.7)
    density = driver.field2data(rho)
    driver.set_density(density)

Iter:  0  - Conv:  0.03712146707277951
Iter:  1  - Conv:  0.0007346997237829209
Iter:  2  - Conv:  1.3115703657018404e-05
Iter:  3  - Conv:  6.056566047603151e-08
Iter:  4  - Conv:  3.062203694233931e-10


In [10]:
driver.calc_energy()

-137.9144917818802

In [11]:
rho.integral()

10.999999999909704

In [12]:
ions = driver.get_dftpy_ions()

In [13]:
rho.write('tmp.xsf', ions=ions)